In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import pandas as pd


In [3]:
df = pd.read_csv('/content/drive/MyDrive/1_10_seasons_tbbt.csv')

In [4]:
df.head()

,episode_name,dialogue,person_scene
0,Series 01 Episode 01 – Pilot Episode,A corridor at a sperm bank.,Scene
1,Series 01 Episode 01 – Pilot Episode,So if a photon is directed through a plane wi...,Sheldon
2,Series 01 Episode 01 – Pilot Episode,"Agreed, what’s your point?",Leonard
3,Series 01 Episode 01 – Pilot Episode,"There’s no point, I just think it’s a good id...",Sheldon
4,Series 01 Episode 01 – Pilot Episode,Excuse me?,Leonard


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54406 entries, 0 to 54405
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   episode_name  54406 non-null  object
 1   dialogue      54404 non-null  object
 2   person_scene  54406 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [6]:
df2 = df.drop(['episode_name'], axis = 1 ,inplace=True)
df2

In [7]:
df2.reset_index(inplace=True)                                              

AttributeError: ignored

In [51]:
df2.head()

AttributeError: ignored

In [19]:
df2.drop(['level_0','index'], axis =1)

,dialogue,person_scene
0,A corridor at a sperm bank.,Scene
1,So if a photon is directed through a plane wi...,Sheldon
2,"Agreed, what’s your point?",Leonard
3,"There’s no point, I just think it’s a good id...",Sheldon
4,Excuse me?,Leonard
...,...,...
54401,"And I with you. Question, are you seeking a r...",Sheldon
54402,What if I were?,Ramona
54403,"Well, that would raise a number of problems. ...",Sheldon
54404,Princeton.,Scene


In [8]:
df.drop(index=df[df['person_scene'] == 'Scene'].index, inplace=True)
df

,dialogue,person_scene
1,So if a photon is directed through a plane wi...,Sheldon
2,"Agreed, what’s your point?",Leonard
3,"There’s no point, I just think it’s a good id...",Sheldon
4,Excuse me?,Leonard
5,Hang on.,Receptionist
...,...,...
54400,"Mmm. No big deal, I enjoy spending time with ...",Ramona
54401,"And I with you. Question, are you seeking a r...",Sheldon
54402,What if I were?,Ramona
54403,"Well, that would raise a number of problems. ...",Sheldon


In [9]:
abc = []
abc = df['person_scene'].tolist()

In [10]:
new_abc = set(abc)
new_abc

{'(Wyatt)',
 '(above)',
 '(acting)',
 '(answering)',
 '(approaching)',
 '(arriving)',
 '(belching)',
 '(burping)',
 '(chanting)',
 '(choking)',
 '(crying)',
 '(dramatically)',
 '(entering)',
 '(enters)',
 '(excitedly)',
 '(inside)',
 '(internally)',
 '(knocking)',
 '(laughing)',
 '(laughs)',
 '(leaving)',
 '(lifting)',
 '(likewise)',
 '(measures)',
 '(mouths)',
 '(off)',
 '(outside)',
 '(pff)',
 '(pointing)',
 '(rapping)',
 '(reading)',
 '(repeatedly)',
 '(returning)',
 '(sarcastically)',
 '(screaming)',
 '(screams)',
 '(shouting)',
 '(signing)',
 '(simultaneously)',
 '(singing)',
 '(sings)',
 '(snarkily)',
 '(sniggering)',
 '(squealing)',
 '(squeals)',
 '(together)',
 '(translating)',
 '(uncomfortable)',
 '(undressing)',
 '(voice)',
 '(voiceover)',
 '(whispering)',
 '(whispers)',
 '(writing)',
 '1',
 '12',
 '2',
 'Abby',
 'Actor',
 'Actress',
 'Adam',
 'Alex',
 'Alex)',
 'Alfred',
 'Alice',
 'Alicia',
 'Alien',
 'All',
 'Amelia',
 'Amy',
 'Amy(off)',
 'Angela',
 'Announcement',
 'Anno

In [11]:
Character =['Amy','Bernadette','Howard','Leonard','Sehldon','Raj','Penny']

In [12]:
!pip -q install transformers

     |████████████████████████████████| 2.8 MB 12.9 MB/s 
     |████████████████████████████████| 3.3 MB 40.4 MB/s 
     |████████████████████████████████| 895 kB 25.6 MB/s 
     |████████████████████████████████| 50 kB 5.6 MB/s 
     |████████████████████████████████| 636 kB 46.2 MB/s 


In [93]:

import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

In [14]:
df.rename(columns= {'person_scene': 'name', 'dialogue':'line'}, inplace=True)

In [15]:
df.reset_index(inplace=True)

In [94]:
df.head(5)

,index,line,name
0,1,So if a photon is directed through a plane wi...,Sheldon
1,2,"Agreed, what’s your point?",Leonard
2,3,"There’s no point, I just think it’s a good id...",Sheldon
3,4,Excuse me?,Leonard
4,5,Hang on.,Receptionist


In [17]:
CHARACTER_NAME = 'Sheldon'

In [18]:
contexted = []

# context window of size 7
n = 10

for i in df[df.name == CHARACTER_NAME].index:
  if i < n:
    continue
  row = []
  prev = i - 1 - n # we additionally substract 1, so row will contain current responce and 10 previous responces  
  for j in range(i, prev, -1):
    row.append(df.line[j])
  contexted.append(row)

columns = ['response', 'context'] 
columns = columns + ['context/' + str(i) for i in range(n - 1)]

data = pd.DataFrame.from_records(contexted, columns=columns)

In [129]:
data = data.dropna()
# texts = data["texts"].tolist()
# texts_encodings = tokenizer(texts, truncation=True, padding=True)

In [130]:
train_df,val_df = train_test_split(data, test_size=0.2 ,random_state = 42)

In [131]:
train_df.head(5)

,response,context,context/0,context/1,context/2,context/3,context/4,context/5,context/6,context/7,context/8
2388,"(Knock, knock, knock) Penny.","Excellent. And just an FYI, as I am the exped...","Oh, damn it. Peer pressure. Fine.","Me, too.",I’m in.,"Well, gentlemen, have you reached a decision?","Well, I’m a Hindu. My religion teaches that i...",And you think you can put up with Sheldon?,Yes.,So you guys are seriously considering this?,You still might get on a magazine.
8499,Yes. So much so that I started to panic.,You did?,I really did think you looked pretty.,"Sheldon, this is silly. I’m not missing anoth...",Not until he stops humping his way up my fami...,"Howie, get off of him.","I know, me, too. But it will be fun to have a...",I sent them a bikini shot of you years ago. T...,Want to take a picture of us and send it to y...,Look at me. Dancing with the prettiest girl a...,Thank you for wearing your heels.
3333,Waterfalls!,We’re getting to him.,I said stop it!,"Yeah, Meemaw did the nasty.",Stop it!,All right. I’ll bet your Meemaw didn’t just h...,Never.,Then let go of the ring and walk away.,I don’t want to hear this.,Think about this. The only way your mother wa...,No! I call no Meemaws.
2389,"(Knock, knock, knock) Penny.","(Knock, knock, knock) Sheldon.","(Knock, knock, knock) Penny.","Excellent. And just an FYI, as I am the exped...","Oh, damn it. Peer pressure. Fine.","Me, too.",I’m in.,"Well, gentlemen, have you reached a decision?","Well, I’m a Hindu. My religion teaches that i...",And you think you can put up with Sheldon?,Yes.
10540,"Wow. I anticipated we’d have problems, but I ...",The kind who almost put a pillow over your fa...,"Well, but what kind of scientists would we be...","Sheldon, maybe living together is a bad idea.",How? You had the whole floor to yourself.,"Well, I didn’t, and it’s your fault.",Really? I slept great.,Miserable and exhausted.,"Good morning. See? I didn’t knock, but it’s f...",Yeah.,"If you ever need a break, the owner of the tr..."


In [132]:
val_df.head(5)

,response,context,context/0,context/1,context/2,context/3,context/4,context/5,context/6,context/7,context/8
2021,"But. Oh, this changes everything.","Yeah, well, before they went out of business,...","No. No, this isn’t right. Our food always com...",Golden Dragon.,What? Wh-Where did my cashew chicken come from?,Szechuan Palace closed two years ago.,Yes. From Szechuan Palace.,I’m afraid so. You know the cashew chicken I ...,More?,"Penny, Penny, I think I know what to do. Shel...",It’s exactly the same…
9242,That’s right. It’s called foreplay. And I cou...,"Yeah, you’ve been going out for years. You ha...",Too slow?,"Hey, I don’t think she’s wrong about you goin...","No, and I’m confused. It’s been nearly 24 hou...",Still haven’t heard from her?,Yeah. I wish I’d fought harder for the rest o...,That carrot was delicious.,Mmm. My pleasure.,Thanks for cooking.,That’s not true. We were just laughing right ...
10798,I’ll be right across the hall. You’ll probabl...,So you think you should keep it?,"Well, Leonard, you know, who should keep this...",So if it breaks there’d be none of it?,It’s one of a kind.,Live long and prosper. Live long and prosper....,"Oh, yeah. The Mr. Spock cuckoo clock.",Oh. Remember when we got this at Comic-Con?,"Yeah, let him do it.",Howard just did that.,Then I’ll check the batteries in the smoke de...
2820,But I could call you a cab or an ambulance.,Ow.,"Well, it seems we’ve reached an impasse.","Well, I can’t drive!",I don’t drive.,"Okay, can you drive me?","Well, assuming you’re correct that your right...","Oh, my god. I got to go to the emergency room.",They’re whimsical because ducks have neither ...,Uh-huh.,The ducks in my tub.
6001,You’re obviously good at what you do.,"No, about my job. I want you to tell me I’m g...",Fine. You have very tiny hands.,Give me a compliment.,Okay.,"All right, Sheldon, there’s only one thing le...","Oh, for heaven’s sake. I did your laundry, I ...",I don’t know.,Humorous. Now will you please present my pape...,I should have sent you to the custom car cove...,"I did it. Had to go to three clothing stores,..."


In [133]:
##Creating dataset for our model .
##Starts here.............................

In [134]:
def construct_conv(row,tokenizer, eos = True):
  flatten = lambda l: [item for sublist in l for item in sublist]
  
  conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id]
                        for x in row]))
  
  conv = flatten(conv)

  return conv

In [135]:

# create dataset suitable for our model
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            for _, row in df.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

In [136]:
## Ends here .....

#Cacheing and sarting of checkpoints


In [137]:
def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)


In [138]:
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


In [139]:
def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


In [140]:


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)

In [141]:

## caching end here..



In [142]:

#MModel build

In [143]:


from transformers import AutoModelWithLMHead, AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-small")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:592: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [144]:

"""
Fine-tuning the library models for language modeling on a text file (GPT, GPT-2, BERT, RoBERTa).
GPT and GPT-2 are fine-tuned using a causal language modeling (CLM) loss while BERT and RoBERTa are fine-tuned
using a masked language modeling (MLM) loss.
"""

# Configs
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [145]:


# Args to allow for easy convertion of python script to notebook
class Args():
    def __init__(self):
        self.output_dir = 'output-small'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'microsoft/DialoGPT-small'
        self.config_name = 'microsoft/DialoGPT-small'
        self.tokenizer_name = 'microsoft/DialoGPT-small'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 4
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'

args = Args()

In [146]:

def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [147]:


# Main runner

def main(df_trn, df_val):
    args = Args()
    
    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    model = AutoModelWithLMHead.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.to(args.device)
    
    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelWithLMHead.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelWithLMHead.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

In [148]:
main(train_df, val_df)


09/04/2021 10:51:28 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:592: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
09/04/2021 10:51:35 - INFO - __main__ -   Training/evaluation parameters <__main__.Args object at 0x7f30b42e3710>
09/04/2021 10:51:35 - INFO - __main__ -   Creating features from dataset file at cached
09/04/2021 10:51:56 - INFO - __main__ -   Saving features into cached file cached/gpt2_cached_lm_512
09/04/2021 10:51:57 - INFO - __main__ -   ***** Running training *****
09/04/2021 10:51:57 - INFO - __main__ -     Num examples = 9182
09/04/2021 10:51:57 - INFO - __main__ -     Num E

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2295 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Iteration:   0%|          | 0/2295 [00:00<?, ?it/s]

09/04/2021 11:31:37 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-3500
09/04/2021 11:31:42 - INFO - __main__ -   Saving optimizer and scheduler states to output-small/checkpoint-3500


RuntimeError: ignored

In [125]:
sum(train_df['context/4'].isnull())


0

In [91]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelWithLMHead.from_pretrained('output-small')

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:592: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
404 Client Error: Not Found for url: https://huggingface.co/output-small/resolve/main/config.json


OSError: ignored